load data

In [1]:
import urllib.request
import json
import numpy as np
import pandas as pd

In [2]:
data_url = 'https://raw.githubusercontent.com/KU81-ComSci-ProjectEnd-GameCard/main/master/Predictor/data/processed_data/quality_run.json'
json_data = list()
with urllib.request.urlopen(data_url) as url:
    run_js = json.loads(url.read().decode())
run = pd.DataFrame(run_js)

In [3]:
decks = run.master_deck.tolist()

build lsa model

In [4]:
def upgrade_lst(lst):
  new_lst = list()
  for item in lst:
    if(item != 'Searing Blow'):
      new_lst.append(item+'+1')
  return new_lst
    
def get_ALL_CARD():
  IC_ATTACK_CARDS = ["Strike_R", "Bash", "Anger", "Body Slam", "Clash", "Cleave", "Clothesline", "Headbutt", "Heavy Blade", "Iron Wave", "Perfected Strike", "Pommel Strike", "Sword Boomerang", "Thunderclap", "Twin Strike", "Wild Strike", "Blood for Blood", "Carnage", "Dropkick", "Hemokinesis", "Pummel", "Rampage", "Reckless Charge", "Searing Blow", "Sever Soul", "Uppercut", "Whirlwind", "Bludgeon", "Feed", "Fiend Fire", "Immolate", "Reaper"]
  IC_SKILL_CARDS = ["Defend_R", "Armaments", "Flex", "Havoc", "Shrug It Off", "True Grit", "Warcry", "Battle Trance", "Bloodletting", "Burning Pact", "Disarm", "Dual Wield", "Entrench", "Flame Barrier", "Ghostly Armor", "Infernal Blade", "Intimidate", "Power Through", "Rage", "Second Wind", "Seeing Red", "Sentinel", "Shockwave", "Spot Weakness", "Double Tap", "Exhume", "Impervious", "Limit Break", "Offering"]
  IC_POWER_CARDS = ["Combust", "Dark Embrace", "Evolve", "Feel No Pain", "Fire Breathing", "Inflame", "Metallicize", "Rupture", "Barricade", "Berserk", "Brutality", "Corruption", "Demon Form", "Juggernaut"]
  IRONCLAD_CARDS = IC_ATTACK_CARDS + IC_SKILL_CARDS + IC_POWER_CARDS

  COLORLESS_CARDS = ["Dramatic Entrance", "Flash of Steel", "Mind Blast", "Swift Strike", "HandOfGreed", "Bite", "Expunger", "Ritual Dagger", "Shiv", "Smite", "Through Violence", "Bandage Up", "Blind", "Dark Shackles", "Deep Breath", "Discovery", "Enlightenment", "Finesse", "Forethought", "Good Instincts", "Impatience", "Jack Of All Trades", "Madness", "Panacea", "PanicButton", "Purity", "Trip", "Apotheosis", "Chrysalis", "Master of Strategy", "Metamorphosis", "Secret Technique", "Secret Weapon", "The Bomb", "Thinking Ahead", "Transmutation", "Violence", "Apparition", "Beta", "Insight", "J.A.X.", "Miracle", "Safety", "Magnetism", "Mayhem", "Panache", "Sadistic Nature", ]

  CURSE_CARDS = ["Ascender s Bane", "Clumsy", "Curse of the Bell", "Decay", "Doubt", "Injury", "Necronomicurse", "Normality", "Pain", "Parasite", "Pride", "Regret", "Shame", "Writhe"]
  return IRONCLAD_CARDS + COLORLESS_CARDS + CURSE_CARDS + upgrade_lst(IRONCLAD_CARDS) + upgrade_lst(COLORLESS_CARDS)

In [5]:
vocab = get_ALL_CARD()

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [7]:
def custom_analyzer(doc):
    return doc
    
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocab, analyzer=custom_analyzer)
tfidf_matrix = tfidf_vectorizer.fit_transform(decks)

c:\users\gaming\appdata\local\programs\python\python39\lib\site-packages\sklearn\feature_extraction\text.py:1375: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [8]:
n_components = 20  # Number of dimensions to reduce to
svd = TruncatedSVD(n_components=n_components)
reduced_matrix = svd.fit_transform(tfidf_matrix.T)

In [9]:
print(reduced_matrix)

[[ 8.74016636e+00 -4.47508518e+00 -2.54973330e+00 ... -4.22828441e-02
  -6.47058225e-01  3.48487778e-01]
 [ 2.28737518e+00 -3.70514218e-01  2.56936987e-01 ...  6.23821507e-03
  -4.71010142e-02 -2.13858882e-01]
 [ 1.44000364e+00 -7.92089229e-02 -4.34628592e-01 ... -5.43844145e-01
   1.14855216e-01  2.47321918e-01]
 ...
 [ 2.29644837e-01  4.28772863e-02  2.28675083e-02 ... -1.67246963e-02
  -6.94517381e-02  2.12562061e-02]
 [ 7.89971406e-02  2.92644538e-02  3.67902939e-02 ... -4.05144776e-02
   1.11681321e-02  1.96593518e-02]
 [ 2.51901130e-02  5.40553767e-02 -1.64737665e-02 ... -4.07624364e-02
  -3.32762747e-03 -6.78829796e-03]]


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(reduced_matrix)
print(similarity_matrix)

[[ 1.          0.78695762  0.51495778 ...  0.57139278  0.46198377
   0.00647977]
 [ 0.78695762  1.          0.65899666 ...  0.78591364  0.57289563
  -0.00808456]
 [ 0.51495778  0.65899666  1.         ...  0.5943299   0.43324398
   0.07691922]
 ...
 [ 0.57139278  0.78591364  0.5943299  ...  1.          0.66431204
   0.20772988]
 [ 0.46198377  0.57289563  0.43324398 ...  0.66431204  1.
   0.28922824]
 [ 0.00647977 -0.00808456  0.07691922 ...  0.20772988  0.28922824
   1.        ]]


In [11]:

df_similarity = pd.DataFrame(similarity_matrix)

# Set the index and column names to be the document numbers
doc_names = [f"{vocab[i]}" for i in range(len(vocab))]
df_similarity.index = doc_names
df_similarity.columns = doc_names

# Create an occurrence matrix based on a similarity threshold
similarity_threshold = 0.5  # Adjust this value as needed
df_occurrence = df_similarity

# Set diagonal values to 0 (a document is not similar to itself in this context)
np.fill_diagonal(df_occurrence.values, 0)

print("Occurrence Matrix:")
print(df_occurrence)

Occurrence Matrix:
                   Strike_R      Bash     Anger  Body Slam     Clash  \
Strike_R           0.000000  0.786958  0.514958   0.410258  0.411205   
Bash               0.786958  0.000000  0.658997   0.766546  0.423421   
Anger              0.514958  0.658997  0.000000   0.427146  0.639485   
Body Slam          0.410258  0.766546  0.427146   0.000000  0.305628   
Clash              0.411205  0.423421  0.639485   0.305628  0.000000   
...                     ...       ...       ...        ...       ...   
Safety+1           0.000000  0.000000  0.000000   0.000000  0.000000   
Magnetism+1        0.137258  0.481363  0.131292   0.456048  0.011019   
Mayhem+1           0.571393  0.785914  0.594330   0.731957  0.365941   
Panache+1          0.461984  0.572896  0.433244   0.529158  0.224499   
Sadistic Nature+1  0.006480 -0.008085  0.076919   0.028437 -0.186293   

                     Cleave  Clothesline  Headbutt  Heavy Blade  Iron Wave  \
Strike_R           0.621060     0.5986

In [12]:
import nbimporter
import save_load_matrix as slm

In [15]:
slm.save('reduced_matrix' , 'card.pkl')